In [5]:
from pyspark.mllib.recommendation import *
import random
from operator import *

In [6]:
userItemFile = sc.textFile("D:/HKUST/Big Data Computing/Project/data/fresh_comp_offline/tianchi_fresh_comp_train_user.txt")
itemFile = sc.textFile("D:/HKUST/Big Data Computing/Project/data/fresh_comp_offline/tianchi_fresh_comp_train_item.txt")

In [7]:
userItemFile.take(5)

['10001082\t285259775\t1',
 '10001082\t4368907\t1',
 '10001082\t4368907\t1',
 '10001082\t53616768\t1',
 '10001082\t151466952\t1']

In [8]:
itemFile.take(5)

['100002303\t3368',
 '100003592\t7995',
 '100006838\t12630',
 '100008089\t7791',
 '100012750\t9614']

In [9]:
def parser1(s):
    temp = s.split("\t")
    return (int(temp[0]),int(temp[1]),int(temp[2]))

def parser2(s):
    temp = s.split("\t")
    return (int(temp[0]),int(temp[1]))

In [10]:
userItem = userItemFile.map(parser1).map(lambda x: ((x[0],x[1]),x[2])).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],x[0][1],x[1]))
item = itemFile.map(parser2).map(lambda x: (x[0],x[1]))

In [11]:
userItem.take(5)

[(10001082, 53616768, 9),
 (10001082, 151466952, 3),
 (10001082, 298397524, 3),
 (10001082, 32104252, 3),
 (10001082, 396795886, 3)]

In [12]:
userItem.count()

397276

In [13]:
item.take(5)

[(100002303, 3368),
 (100003592, 7995),
 (100006838, 12630),
 (100008089, 7791),
 (100012750, 9614)]

In [14]:
item.count()

620918

In [15]:
trainData,validationData,testData = userItem.randomSplit([6,2,2], 13)
print(trainData.take(3))
print(validationData.take(3))
print(testData.take(3))
print(trainData.count())
print(validationData.count())
print(testData.count())
trainData.cache()
validationData.cache()
testData.cache()

[(10001082, 151466952, 3), (10001082, 298397524, 3), (10001082, 22667958, 3)]
[(10001082, 53616768, 9), (10001082, 32104252, 3), (10001082, 125083630, 1)]
[(10001082, 396795886, 3), (10001082, 275221686, 34), (10001082, 97441652, 1)]
237839
79807
79630


PythonRDD[28] at RDD at PythonRDD.scala:53

In [17]:
def modelEval(model,validation):
    import operator
    userIds = validation.map(lambda x: x[0]).distinct().collect()
    itemIds = list(item.collectAsMap().keys())
    temp = trainData.map(lambda x: (x[0], x[1])).filter(lambda x: x[0] in userIds).groupByKey().map(lambda x: (x[0], list(x[1])))
    valiSet = temp.flatMap(lambda x: [(x[0],a) for a in itemIds if a not in x[1]])
    predictions = bestModel.predictAll(valiSet).map(lambda x: (x[0], x[1], x[2])).groupBy(lambda x:x[0]).map(lambda x:(x[0],sorted(list(x[1]),key=operator.itemgetter(2),reverse=True)))
    predictions = predictions.map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    cmprSet = validation.groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1]))).map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    p = predictions.collect()
    cdic =  cmprSet.collectAsMap()  
    scores = []
    for i in range(0,len(p)):
        user = p[i][0]
        plist = p[i][1]
        clist = cdic[user]
        plen = len(plist)
        clen = len(clist)
        count = 0
        score = 0
        if plen >= clen:
            for j in range(0,clen):
                if plist[j] in clist:
                    count += 1
        score = float(float(count)/clen)
        scores.append(score)
    return sum(scores)/len(scores)

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
score = 0
best_rank = 2
for elmt in range(2,11,2):
    bestModel = ALS.trainImplicit(trainData, rank=elmt)
    best_model_score = modelEval(bestModel,validationData)
    if model_score > score:
        score = best_model_score
        best_rank = elmt

print('The best model score for rank ' + str(best_rank) + ' is '+ str(best_model_score))